In [1]:
%load_ext autoreload
%autoreload 2

from aiida import orm, engine, load_profile
from aiida.common import NotExistent

# import aiida_dislocation
# import importlib
# importlib.reload(aiida_dislocation.workflows.usf)
from aiida_dislocation.workflows.usf import USFWorkChain

from ase.io import read
from ase.build import bulk
from aiida.engine import run

import logging


load_profile()

Profile<uuid='251a7c0fb83e42718a3a51e0ee33ee7a' name='lucia'>

In [2]:


group_label = 'test/workchains/USF/'
test_USF_group, created = orm.Group.collection.get_or_create(label=group_label)
if created:
    print(f"Group '{group_label}' created.")
else:
    print(f"Group '{group_label}' already exists.")


computer = 'lucia'
lucia_computer = orm.load_computer(computer)
lucia_computer.set_workdir('/gpfs/scratch/acad/elph/yimzhang/aiida_workdir/test_aiida_dislocation')

version = '73'
code = 'pw'
label = f'qe-{version}-{code}'
bin_dir = '/gpfs/scratch/acad/elph/yimzhang/apps/q-e-qe-7.3/bin/'
try:
    orm.load_code(f'{label}@{computer}')
except NotExistent as exc:
    orm.InstalledCode(
        label=label,
        description=f'Quantum ESPRESSO {version} version for {code}.x',
        computer=lucia_computer,
        filepath_executable=f'{bin_dir}/{code}.x',
        default_calc_job_plugin=f'quantumespresso.{code}',
    ).store()

pw_label = f'qe-{version}-pw'
pw_code = orm.load_code(f'{pw_label}@{computer}')


Group 'test/workchains/USF/' already exists.


# TEST A1 -- Aluminium

In [3]:
import yaml

ase_atom = read('./structures/cif/Al-225-A1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf-lucia.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


## {100} surface

In [4]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '100', [0.0, 0.5]])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)
logging.getLogger('aiida.broker.rabbitmq').setLevel(logging.ERROR)
test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/09/2025 07:52:19 PM <230043> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138226|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/09/2025 07:52:19 PM <230043> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138226|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
05/09/2025 07:52:19 PM <230043> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138226|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. 
05/09/2025 07:52:19 PM <230043> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138226|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.0, 0.5]. 
05/09/2025 07:52:19 PM <230043> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138226|USFWorkChain|run_scf]: launching PwCalculatio

## {110} surface

In [5]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '110', [0.0, 0.5]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/09/2025 08:07:56 PM <235524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138275|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/09/2025 08:07:56 PM <235524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138275|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/09/2025 08:07:57 PM <235524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138275|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. 
05/09/2025 08:07:57 PM <235524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138275|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.0, 0.5]. 
05/09/2025 08:07:57 PM <235524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138275|USFWorkChain|run_scf]: launching PwCalculatio

## {111} surface

In [7]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(9)
inputs['slipping_system'] = orm.List(['A1', '111', None])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: We are doing FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: ABC stacking for {111} gliding plane. Faulting possible. slipping direction is along 1/2[110].
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_scf]: launching PwCalculation<71585> for Al unit cell geometry.
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|inspect_scf]: SCF calculation<71585> finished
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_USF]: Multiplicity of the faulted geometr

# TEST A2 -- Vanadium

In [5]:
import yaml

ase_atom = read('./structures/cif/V-227-A2.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([14, 14, 14])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf-lucia.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-6



## {100} surface

In [5]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(16)
inputs['slipping_system'] = orm.List(['A2', '100'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 03:17:04 PM <885922> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71900|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
04/28/2025 03:17:04 PM <885922> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71900|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
04/28/2025 03:17:04 PM <885922> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71900|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. Slipping direction: 1/2[110]. 
04/28/2025 03:17:04 PM <885922> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71900|USFWorkChain|run_scf]: launching PwCalculation<71901> for V unit cell geometry.
04/28/2025 03:17:35 PM <885922> aiida.transport.SshTransport: [ERROR] Error connecting to proxy 'gwceci.cism.ucl.ac.be' through SSH: [SshTransp

KilledError: Process was killed because the runner received an interrupt

04/28/2025 03:19:05 PM <885922> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/home/ucl/modl/yimzhang/miniforge3/envs/aiida-240/lib/python3.11/site-packages/aiida/engine/transports.py", line 106, in request_transport
    yield transport_request.future
  File "/home/ucl/modl/yimzhang/miniforge3/envs/aiida-240/lib/python3.11/site-packages/aiida/engine/processes/calcjobs/tasks.py", line 86, in do_upload
    transport = await cancellable.with_interrupt(request)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ucl/modl/yimzhang/miniforge3/envs/aiida-240/lib/python3.11/site-packages/aiida/engine/utils.py", line 115, in with_interrupt
    result = await next(wait_iter)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ucl/modl/yimzhang/miniforge3/envs/aiida-240/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
plumpy.process_s

## {110} surface

In [6]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A2', '110'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)
logging.getLogger('aiida.broker.rabbitmq').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/09/2025 07:10:03 PM <213793> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.9.27 is not supported and will cause unexpected problems!
05/09/2025 07:10:03 PM <213793> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
05/09/2025 07:10:03 PM <213793> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
05/09/2025 07:10:04 PM <213793> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138169|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
05/09/2025 07:10:04 PM <213793> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138169|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/09/2025 07:10:04 PM <213793> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138169|USFWorkChain|validate_slip

# TEST B1 -- AsTe

In [29]:
import yaml

ase_atom = read('./structures/cif/mpds-S1605628-AsTe-225-B1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([14, 14, 14])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf-lucia.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}


inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


## {100} surface

In [30]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['B1', '100'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 07:44:34 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137980|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype NaCl>. I don't know the usual gliding plane. 
04/28/2025 07:44:35 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137980|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
04/28/2025 07:44:35 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137980|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. Slipping direction: 1/2[010]. 
04/28/2025 07:44:35 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137980|USFWorkChain|run_scf]: launching PwCalculation<1137981> for AsTe unit cell geometry.
04/28/2025 07:45:49 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137980|USFWorkChain|inspec

## {110} surface

In [33]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['B1', '110'])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 08:10:56 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138008|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype NaCl>. I don't know the usual gliding plane. 
04/28/2025 08:10:56 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138008|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
04/28/2025 08:10:56 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138008|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. Slipping direction: 1/2[112]. 
04/28/2025 08:10:57 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138008|USFWorkChain|run_scf]: launching PwCalculation<1138009> for AsTe unit cell geometry.
04/28/2025 08:12:11 PM <139415> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138008|USFWorkChain|inspec

## {111} surface

# TEST HALF-HEUSLER

In [12]:

import yaml

ase_atom = read('./structures/cif/mpds-S1612207-NbCoSb-216-C1b.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf-lucia.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5



## {100} surface

In [14]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['C1b', '100'])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)

04/29/2025 11:00:02 AM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138093|USFWorkChain|validate_slipping_system]: We are doing half-heusler-type structure. <space group #216, prototype MgSiAl>. I don't know the usual gliding plane. 
04/29/2025 11:00:02 AM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138093|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
04/29/2025 11:00:02 AM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138093|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is ABCD. Slipping direction: 1/2[100]. 
04/29/2025 11:00:02 AM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138093|USFWorkChain|run_scf]: launching PwCalculation<1138094> for CoNbSb unit cell geometry.
04/29/2025 11:02:52 AM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] 

# {110} surface

In [17]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['C1b', '110'])
inputs['pw']['parameters']['ELECTRONS']['mixing_beta'] = 0.4

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)

04/29/2025 01:05:11 PM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138145|USFWorkChain|validate_slipping_system]: We are doing half-heusler-type structure. <space group #216, prototype MgSiAl>. I don't know the usual gliding plane. 
04/29/2025 01:05:11 PM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138145|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
04/29/2025 01:05:11 PM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138145|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. Slipping direction: 1/2[110]. 
04/29/2025 01:05:12 PM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1138145|USFWorkChain|run_scf]: launching PwCalculation<1138146> for CoNbSb unit cell geometry.
04/29/2025 01:06:27 PM <293207> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1